In [1]:
# 爬虫
import sqlite3
import sys
from pprint import pprint
import webbrowser
import subprocess
import random
import pandas as pd
import numpy as np
import requests
import re

# 词云
import  jieba   #jieba分词包  
from PIL import Image #图片的处理
import matplotlib.pyplot as plt # 主要进行词云绘图
from wordcloud import WordCloud  #词云
import matplotlib.colors as colors
from collections import Counter
import xlwt
import docx

In [2]:
urls=pd.read_excel('谷物麦片_test.xls')['笔记链接']
urls.head()

0    https://www.xiaohongshu.com/discovery/item/5f9...
1    https://www.xiaohongshu.com/discovery/item/5f9...
2    https://www.xiaohongshu.com/discovery/item/5f9...
3    https://www.xiaohongshu.com/discovery/item/5f9...
4    https://www.xiaohongshu.com/discovery/item/5fa...
Name: 笔记链接, dtype: object

In [3]:
#def pyGoogleSearch(url):
#    for i in range(len(urls)):
#        webbrowser.open(urls[i])
#        input('Hit return to continue to next record')

In [4]:
# pyGoogleSearch(urls)

In [1]:
info={
    "User-Agent":"Mozilla/5.0 (Macintosh; Intel Mac OS X 10.15; rv:87.0) Gecko/20100101 Firefox/87.0",
    "Cookie":"_data_chl=key=BaiduOrginal; Hm_lvt_c6d9cdbaf0b464645ff2ee32a71ea1ae=1618112965,1618112967,1618206527,1618365003; Hm_lvt_ed4e8df159c27d4f0ce1a965612b94dd=1615947686,1615951323,1615960564; Hm_lpvt_c6d9cdbaf0b464645ff2ee32a71ea1ae=1618365003; Hm_lpvt_ed4e8df159c27d4f0ce1a965612b94dd=1618365366; User=UserId=3299e1743fb236c7&Password=1599909c62eb6f2b26f7add5fa56ce27&ChildId=2126"
}

### Test with single URL

In [6]:
test_url=urls[0]
test_data = requests.get(test_url, headers = info).text

In [7]:
test_data

'<!doctype html><html><head><meta charset="utf-8"><meta name="description" content="我家中常备的就是麦片啦！早上可以泡牛奶做早餐吃 下午可以整点花式下午茶 当作晚餐吃饱腹又代餐 全能选手没错啦～ 最近入手了新麦片尝尝鲜 周末在家做了高颜值的猕猴桃燕麦杯当下午茶吃 有颜又好吃 - 也给"><meta name="keywords" content="麦片,燕麦,猕猴桃,桃仁,谷物麦片,水果燕麦,早餐,夏威夷果,酸奶,代餐,谷物,美食,美食教程"><meta name="shenma-site-verification" content="3abbfaac4345ca4daaa5ad2282115298_1476771175"><meta name="360-site-verification" content="86dcc68a445e2ed8034e85f0ba88a83a"><meta name="sogou-site-verification" content="s2WSHhCpCV"><meta name="google-site-verification" content="jxL_cAIumLKHmN3VWfOMPoXtw_7nK1pZZRdJRYMC0oA"><meta name="viewport" content="width=device-width,initial-scale=1,minimum-scale=1,maximum-scale=1,user-scalable=no,viewport-fit=cover"><meta name="format-detection" content="telephone=no,address=no,email=no"><meta name="mobileOptimized" content="width"><meta name="HandheldFriendly" content="true"><meta name="applicable-device" content="pc,mobile"><meta http-equiv="X-UA-Compatible" content="IE=Edge,chrome=1"><me

In [5]:
test_content = re.compile(',"title":"(.*?)","id":')
test_content_list = test_content.findall(test_data)

In [8]:
test_content_list

['宅家自制水果燕麦杯！健康谷物麦片太可了～","desc":"我家中常备的就是麦片啦！早上可以泡牛奶做早餐吃 下午可以整点花式下午茶 当作晚餐吃饱腹又代餐👀 全能选手没错啦～\\n最近入手了新麦片尝尝鲜 周末在家做了高颜值的猕猴桃燕麦杯当下午茶吃 🌈有颜又好吃 \\n-\\n👉🏻也给大家分享我的麦片杯做法哦\\n1⃣️先在杯底铺上一层麦片\\n️2⃣️把切片的猕猴桃固定在杯壁上一圈\\n3⃣️倒入酸奶后顶层再铺上麦片，最后用薄荷叶点缀下\\n-\\n简单几步手残也能Get同款哦！🥳我用的是欣善怡家的麦片做的燕麦杯 🤍是来源于新西兰进口的麦片 包含多种谷物🥖和果干 🍒营养丰富好吃不会胖！\\n-\\n💛燕麦夏威夷果口味：里面有香脆好吃的夏威夷果哦！还添加了膨化小麦，吃起来像爆米花一样松脆，越吃越上瘾[哇R]\\n💙椰果扁桃仁果味：椰果和扁桃仁的神仙组合！干吃或者搭配酸奶、牛奶一起吃都好吃 清爽不油腻 脆爽可口～[斜眼R]\\n-\\n没有什么比这样一碗好吃又不胖的燕麦杯更诱人的啦！专治嘴馋，可要家中常囤啦！👻']

### Test with short lists of URL (using for loop)

In [31]:
temp_content=re.compile(',"title":"(.*?)","id":')
temp_data=[requests.get(urls[0], headers = info).text]
temp_content_list=[temp_content.findall(temp_data[0])]
temp_urls=[urls[0],urls[1],urls[2],urls[3]]
for i in range(1,len(temp_urls)):
    temp_data.append(requests.get(temp_urls[i],headers = info).text)
    temp_content_list.append(temp_content.findall(temp_data[i]))

In [32]:
temp_content_list

[['宅家自制水果燕麦杯！健康谷物麦片太可了～","desc":"我家中常备的就是麦片啦！早上可以泡牛奶做早餐吃 下午可以整点花式下午茶 当作晚餐吃饱腹又代餐👀 全能选手没错啦～\\n最近入手了新麦片尝尝鲜 周末在家做了高颜值的猕猴桃燕麦杯当下午茶吃 🌈有颜又好吃 \\n-\\n👉🏻也给大家分享我的麦片杯做法哦\\n1⃣️先在杯底铺上一层麦片\\n️2⃣️把切片的猕猴桃固定在杯壁上一圈\\n3⃣️倒入酸奶后顶层再铺上麦片，最后用薄荷叶点缀下\\n-\\n简单几步手残也能Get同款哦！🥳我用的是欣善怡家的麦片做的燕麦杯 🤍是来源于新西兰进口的麦片 包含多种谷物🥖和果干 🍒营养丰富好吃不会胖！\\n-\\n💛燕麦夏威夷果口味：里面有香脆好吃的夏威夷果哦！还添加了膨化小麦，吃起来像爆米花一样松脆，越吃越上瘾[哇R]\\n💙椰果扁桃仁果味：椰果和扁桃仁的神仙组合！干吃或者搭配酸奶、牛奶一起吃都好吃 清爽不油腻 脆爽可口～[斜眼R]\\n-\\n没有什么比这样一碗好吃又不胖的燕麦杯更诱人的啦！专治嘴馋，可要家中常囤啦！👻'],
 ['高颜值红薯谷物牛奶杯🥤，冬日热饮get","desc":"🌿红薯直接蒸着吃腻了，那就来点不一样的，简单几步DIY，就是一杯有颜有料的冬日热饮啦🌿 @吃货薯 \\n食材：蒸熟的红薯150克、奶粉100克、谷物麦片适量 @生活薯 \\n\xa0👇🏻\xa0步骤\xa0👇🏻\\n1⃣️养生壶烧水，水烧开后倒400ml把奶粉泡开\\n2⃣️红薯倒入大宇轻养破壁机中，倒入100ml泡好的牛奶，用奶昔功能打成红薯泥\\n3⃣️用勺子把红薯泥填入杯中\\n4⃣️剩下的热牛奶打发出奶泡，倒入杯子中\\n5⃣️最后撒上谷物麦片就🆗了 @吃不饱同学 \\n💕喝之前用勺子搅拌均匀，香甜美味又温暖哦💕 @哥登生活好物 '],
 ['","desc":"卡乐比麦片800克，真正日本本土版，营养早餐，吃法多多！！香脆可口！[色]Calbee卡乐比🍥燕麦水果谷物麦片（无添加防腐剂），多种水果谷物：🍎苹果 、🏉木瓜、🍓草莓 、🌴椰果、🎃南瓜仁 、🍇葡萄干、🌾燕麦、🍚玄米、🍀小麦，富含维他命A、D、B1、B6、B12等，营养美味，泡牛奶、冲豆浆、配酸奶、拌沙拉，可以方便简单也可高大上！尽情享用吧😊'],
 ['Day187 低卡饱腹颜在线的｜香蕉树梅燕麦粥","desc":"早啊

### Get full content lists (using for loop) 

In [34]:
content=re.compile(',"title":"(.*?)","id":')
data=[requests.get(urls[0], headers = info).text]
content_list=[content.findall(data[0])]
for i in range(1,len(urls)):
    data.append(requests.get(urls[i],headers = info).text)
    content_list.append(content.findall(data[i]))

In [35]:
content_list

[['宅家自制水果燕麦杯！健康谷物麦片太可了～","desc":"我家中常备的就是麦片啦！早上可以泡牛奶做早餐吃 下午可以整点花式下午茶 当作晚餐吃饱腹又代餐👀 全能选手没错啦～\\n最近入手了新麦片尝尝鲜 周末在家做了高颜值的猕猴桃燕麦杯当下午茶吃 🌈有颜又好吃 \\n-\\n👉🏻也给大家分享我的麦片杯做法哦\\n1⃣️先在杯底铺上一层麦片\\n️2⃣️把切片的猕猴桃固定在杯壁上一圈\\n3⃣️倒入酸奶后顶层再铺上麦片，最后用薄荷叶点缀下\\n-\\n简单几步手残也能Get同款哦！🥳我用的是欣善怡家的麦片做的燕麦杯 🤍是来源于新西兰进口的麦片 包含多种谷物🥖和果干 🍒营养丰富好吃不会胖！\\n-\\n💛燕麦夏威夷果口味：里面有香脆好吃的夏威夷果哦！还添加了膨化小麦，吃起来像爆米花一样松脆，越吃越上瘾[哇R]\\n💙椰果扁桃仁果味：椰果和扁桃仁的神仙组合！干吃或者搭配酸奶、牛奶一起吃都好吃 清爽不油腻 脆爽可口～[斜眼R]\\n-\\n没有什么比这样一碗好吃又不胖的燕麦杯更诱人的啦！专治嘴馋，可要家中常囤啦！👻'],
 ['高颜值红薯谷物牛奶杯🥤，冬日热饮get","desc":"🌿红薯直接蒸着吃腻了，那就来点不一样的，简单几步DIY，就是一杯有颜有料的冬日热饮啦🌿 @吃货薯 \\n食材：蒸熟的红薯150克、奶粉100克、谷物麦片适量 @生活薯 \\n\xa0👇🏻\xa0步骤\xa0👇🏻\\n1⃣️养生壶烧水，水烧开后倒400ml把奶粉泡开\\n2⃣️红薯倒入大宇轻养破壁机中，倒入100ml泡好的牛奶，用奶昔功能打成红薯泥\\n3⃣️用勺子把红薯泥填入杯中\\n4⃣️剩下的热牛奶打发出奶泡，倒入杯子中\\n5⃣️最后撒上谷物麦片就🆗了 @吃不饱同学 \\n💕喝之前用勺子搅拌均匀，香甜美味又温暖哦💕 @哥登生活好物 '],
 ['","desc":"卡乐比麦片800克，真正日本本土版，营养早餐，吃法多多！！香脆可口！[色]Calbee卡乐比🍥燕麦水果谷物麦片（无添加防腐剂），多种水果谷物：🍎苹果 、🏉木瓜、🍓草莓 、🌴椰果、🎃南瓜仁 、🍇葡萄干、🌾燕麦、🍚玄米、🍀小麦，富含维他命A、D、B1、B6、B12等，营养美味，泡牛奶、冲豆浆、配酸奶、拌沙拉，可以方便简单也可高大上！尽情享用吧😊'],
 ['Day187 低卡饱腹颜在线的｜香蕉树梅燕麦粥","desc":"早啊

### 词云分析

In [50]:
fullContent=pd.DataFrame(content_list,columns=['内容'])
fullContent

,内容
0,"宅家自制水果燕麦杯！健康谷物麦片太可了～"",""desc"":""我家中常备的就是麦片啦！早上可以..."
1,"高颜值红薯谷物牛奶杯🥤，冬日热饮get"",""desc"":""🌿红薯直接蒸着吃腻了，那就来点不一..."
2,""",""desc"":""卡乐比麦片800克，真正日本本土版，营养早餐，吃法多多！！香脆可口！[色..."
3,"Day187 低卡饱腹颜在线的｜香蕉树梅燕麦粥"",""desc"":""早啊～\n昨天晚上去吃了羊..."
4,"宿舍女孩减脂必囤✨一起变美变瘦吧"",""desc"":""嘿嘿嘿❗低卡零食分享收好啦✨\n都是平价..."
5,"🥣菊乐酸奶➕Nana谷物麦片"",""desc"":""最近真的超爱这款菊乐酸奶！！太好喝了！！以前..."
6,"酸酸甜甜嚼着喝的果粒酸奶来啦!"",""desc"":""惠丰嚼酸奶，小清新的颜值,\n打开是一股浓..."
7,"11月3日早餐｜吐司版披萨🍕"",""desc"":""☀️早安🏠早餐\n感恩的心\n都藏在每一餐里..."
8,"懒人厨房｜那些独自宅家的日子🌈燕麦为伴"",""desc"":""[汗颜R][失望R]老爸老妈去旅游..."
9,"坚持护发养发，不做秃头小宝贝"",""desc"":""我平时很喜欢做发型，还隔三差五卷一次头发\n..."


In [51]:
neiRong=fullContent['内容'][0]
for i in range(1,len(fullContent['内容'])):
    neiRong=neiRong+'\n'+fullContent['内容'][i]

In [53]:
# 定义函数
def fenJu(text):
    juzi=[]
    start=0
    end=0
    for i in range(len(text)):
        if text[i]=='\n':
            end=i+1
            juzi.append(text[start:end])
            start=i+1
    return juzi

def tiQu(juzi,ci):
    output=''
    count=0
    for i in range(len(juzi)):
        if juzi[i].find(ci)>=0:
            output=output+str(count+2)+'.'+juzi[i]
            count=count+1
    return count,output

def multiTiqu(aList, diZhi):
    for i in aList:
        output=tiQu(juzi,i)
        #创建Word文档
        doc = docx.Document()
        #写入Word文档
        doc.add_paragraph(output[1])
        #保存Word文档
        doc.save(diZhi+'%d' % output[0]+'-'+i+".docx")
        
def yiLeiCi(juzi,aList,diZhi):
    output=''
    count=0
    aList=['油','贵']
    for i in range(len(juzi)):
        for j in range(len(aList)):
            if juzi[i].find(aList[j])>=0:
                output=output+str(count+2)+'.'+juzi[i]
                count=count+1
                break
    #创建Word文档
    doc = docx.Document()
    #写入Word文档
    doc.add_paragraph(output)
    #保存word文档
    name = aList[0]
    for i in range(1,len(aList)):
        name = name + '&' + aList[i]
    doc.save(diZhi+'%d' % count+'-'+name+".docx")

In [56]:
wordlist = list(jieba.cut(neiRong))
wordlist = [word for word in wordlist if len(word)>1]

Building prefix dict from the default dictionary ...
Loading model from cache /var/folders/0x/pbr2zq2j1qjdqkfqx5l5_m9w0000gn/T/jieba.cache
Loading model cost 0.975 seconds.
Prefix dict has been built successfully.


In [60]:
wordlist
word = " ".join(wordlist)

counts={}
for i in wordlist:
    counts[i]=counts.get(i,1)+1

items=list(counts.items())
def takeSecond(elem):
    return elem[1]

items.sort(key=takeSecond,reverse=True) 

itemsDict = {}
for i in range(len(items)):
    itemsDict[items[i][0]] = items[i][1]

cipin = pd.DataFrame.from_dict(itemsDict, orient='index')

writer=pd.ExcelWriter('/Users/littletrista/Desktop/PE/爬虫/谷物麦片.xlsx')
cipin.to_excel(writer,sheet_name='词频')
writer.save()